### Interpretability/Explainability framework with EvalxNLP

Documentation: [Github](https://github.com/dmah10/EvalxNLP)

Original Paper: [Dhaini et al 2025](https://arxiv.org/abs/2505.01238)

Key Tutorial: [Tutorial NLI](https://github.com/dmah10/EvalxNLP/blob/main/examples/tutorial_nli.ipynb)


Install instructions:

git clone https://github.com/dmah10/EvalxNLP.git

cd EvalxNLP

pip install -r requirements.txt

In [2]:
from EvalxNLP import XAIFramework



ImportError: cannot import name 'XAIFramework' from 'EvalxNLP' (unknown location)